In [ ]:
import os
from PIL import Image

# Function to resize an image
def resize_image(input_path, output_path, size):
    with Image.open(input_path) as img:
        img_resized = img.resize(size)
        img_resized.save(output_path)

# Function to create missing size folders and resize images
def process_size_folder(input_size_folder, output_size_folder, size):
    os.makedirs(output_size_folder, exist_ok=True)
    for file in os.listdir(input_size_folder):
        if file.endswith('.jpg') or file.endswith('.png'):  # Adjust file formats as necessary
            input_path = os.path.join(input_size_folder, file)
            output_path = os.path.join(output_size_folder, file)
            resize_image(input_path, output_path, size)

# Function to process images in each folder
def process_folder(input_folder, output_folder):
    for folder in os.listdir(input_folder):
        input_folder_path = os.path.join(input_folder, folder)
        if os.path.isdir(input_folder_path):
            output_folder_path = os.path.join(output_folder, folder)
            os.makedirs(output_folder_path, exist_ok=True)
            
            # Create missing size folders if necessary
            for size in ['small', 'medium', 'large']:
                size_folder_path = os.path.join(input_folder_path, size)
                if not os.path.exists(size_folder_path):
                    os.makedirs(size_folder_path)
                    # Resize images in missing size folders if the 'large' folder exists
                    if size == 'medium' or size == 'small':
                        large_folder_path = os.path.join(input_folder_path, 'large')
                        if os.path.exists(large_folder_path):
                            for file in os.listdir(large_folder_path):
                                input_path = os.path.join(large_folder_path, file)
                                output_path = os.path.join(size_folder_path, file)
                                resize_dimensions = {'small': (200, 200), 'medium': (540, 540)}
                                resize_image(input_path, output_path, resize_dimensions[size])
            
            # Resize images based on available size folders
            for size in ['small', 'medium', 'large']:
                process_size_folder(os.path.join(input_folder_path, size), os.path.join(output_folder_path, size), {'small': (200, 200), 'medium': (540, 540), 'large': (1080, 1080)}[size])

def resize_image(input_path, output_path, size):
    try:
        img = Image.open(input_path)
        img.thumbnail(size)
        img.save(output_path)
    except Exception as e:
        print(f"Error resizing image {input_path}: {e}")

def process_images(root_folder):
    for foldername in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, foldername)

        # Create large, medium, and small folders if they don't exist
        for img_folder in ["large", "medium", "small"]:
            img_folder_path = os.path.join(folder_path, img_folder)
            if not os.path.exists(img_folder_path):
                os.makedirs(img_folder_path)

        # Get image paths from any of the folders
        image_folders = ["large", "medium", "small"]
        image_files = []
        for img_folder in image_folders:
            img_folder_path = os.path.join(folder_path, img_folder)
            if os.path.exists(img_folder_path):
                image_files.extend([os.path.join(img_folder_path, img) for img in os.listdir(img_folder_path) if os.path.isfile(os.path.join(img_folder_path, img))])

        # Resize images and save to large, medium, and small folders
        for img_path in image_files:
            img_name, img_ext = os.path.splitext(os.path.basename(img_path))

            # Resize and save to large folder
            large_output_path = os.path.join(folder_path, "large", f"{img_name}{img_ext}")
            resize_image(img_path, large_output_path, (1080, 1080))

            # Resize and save to medium folder
            medium_output_path = os.path.join(folder_path, "medium", f"{img_name}{img_ext}")
            resize_image(img_path, medium_output_path, (540, 540))

            # Resize and save to small folder
            small_output_path = os.path.join(folder_path, "small", f"{img_name}{img_ext}")
            resize_image(img_path, small_output_path, (200, 200))

if __name__ == "__main__":
    input_folder = r'C:\s3copydse'
    output_folder = r'C:\s4'
    # Process folders
    process_folder(input_folder, output_folder)
    print("Image resizing completed.")
    root_folder = r"C:\s4"
    process_images(root_folder)
    print("Images resized and saved successfully!")